In [ ]:
import os
import time
import json
import numpy as np
import pandas as pd
import threading

from scipy.interpolate import interp1d
from scipy import signal

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

from mpl_finance import candlestick_ohlc
import mpl_finance as mplf

import tkinter as tk
from tkinter import *
from tkinter import ttk

import tensorflow as tf



In [ ]:
file_names = os.listdir("D:\\market_train\\dataset\\nifty 1m")


file_name = "ASIANPAINT_with_indicators_.csv"
csv_data = pd.read_csv("D:\\market_train\\dataset\\data creation\\"+file_name)
csv_data['date'] = pd.to_datetime(csv_data['date'])


In [ ]:
def normalize(data, window_limit):
    data = np.array(data,dtype=np.float32)

    max_value = max(data[0:window_limit])
    min_value = min(data[0:window_limit])
    if min_value != max_value:
        normalized_data = (data - min_value) / (max_value - min_value)

        min_value = min(normalized_data[0:window_limit])
        max_value = max(normalized_data[0:window_limit])

        normalized_data = (normalized_data - min_value) / (max_value - min_value)

        return normalized_data,True
    else:
        return False,False
    
def low_pass_filter(data):

    sampling_frequency = 1.0    
    nyquist_frequency = 0.5 * sampling_frequency
    cutoff_frequency = 0.4
    order = int(len(data)/6)+1
    #print(order)
    try:
        b, a = signal.butter(order, cutoff_frequency, btype='low')
        filtered_data = signal.filtfilt(b, a, data)
    except:
        return False,False
    return filtered_data,True

def interpolate_sequence(data, target_size):
    

    x = np.arange(len(data))
    f = interp1d(x, data, kind='cubic')
    new_x = np.linspace(0, len(data) - 1, target_size)
    interpolated_data = f(new_x)
    
    return interpolated_data.tolist()

def invert_sequence(data):
    data = np.array(data)
    max_value = max(data)
    inverted_data = [max_value - x for x in data]+max_value
    return inverted_data - 1



In [ ]:
class Tree:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.left = None
        self.right = None


class Write_to_csv:
    
    def __init__(self) -> None:
        global counter
        self.current_pattern = None
        self.start_index = None
        self.shoulder1_index = None
        self.shoulder2_index = None
        self.neckline_index = None
        self.neckline1_index = None
        self.neckline2_index = None
        self.head_index = None
        self.top1_index = None
        self.top2_index = None
        self.target_index = None
        self.price_counter = 0
        self.changes = None
        self.save_data_thread = threading.Thread(target=self.save_data_to_csv)
        self.save_data_thread.start()
        self.start_time = time.time()
        self.current_state = {}
        with open('current_status.json','r') as json_status_file:
            self.current_state = json.load(json_status_file)
        counter = self.current_state['completed index']
        self.previous_counter = counter

        process_graph.update_plot(0)
        Label_changes.show_counter()




    def save_data_to_csv(self):
        while True:
            time.sleep(60)
            self.current_state['completed index'] = counter
            self.current_state['total time spent'] += time.time() - self.start_time
            self.current_state['total completed index'] += counter - self.previous_counter
            self.start_time = time.time()
            self.previous_counter = counter


            with open('current_status.json','w') as json_status_file:
                json.dump(self.current_state,json_status_file)

            if self.changes != None:
                csv_data.to_csv('D:\\market_train\\dataset\\data creation\\'+file_name, index=False )
                print("saved changes")
                self.changes = None
            else:
                print("no changes saved")
            

    def write_double_top_to_csv(self,value1,value2,value3,value4,value5):
        if self.current_pattern == "DOUBLE TOP":
            self.changes = 1
            self.start_index = value1 + counter
            self.top1_index = value2 + counter
            self.neckline_index = value3 + counter
            self.top2_index = value4 + counter
            self.target_index = value5 + counter


            for index in range(self.start_index,self.target_index+1,1):
                csv_data.at[index,'patternName_doubletop'] = 'DT'

            csv_data.at[self.start_index,'patternFeatures_doubletop'] = 'START'
            csv_data.at[self.top1_index,'patternFeatures_doubletop'] = 'TOP 1'
            csv_data.at[self.neckline_index,'patternFeatures_doubletop'] = 'NECKLINE'
            csv_data.at[self.top2_index,'patternFeatures_doubletop'] = 'TOP 2'
            csv_data.at[self.target_index,'patternFeatures_doubletop'] = 'TARGET'

            self.current_state['total pattern identified'] += 1
            self.current_state['total double top'] += 1

            #csv_data.to_csv('D:\\market_train\\dataset\\temp_data_creation_updated.csv')


        print(self.start_index,
              self.top1_index,
              self.neckline_index,
              self.top2_index,
              self.target_index,
              self.current_pattern)
        
    def write_head_and_shoulders_to_csv(self,value1,value2,value3,value4,value5,value6,value7):
        if self.current_pattern == "HEAD AND SHOULDERS":
            self.changes = 1
            self.start_index = value1 + counter
            self.shoulder1_index = value2 + counter
            self.neckline1_index = value3 + counter
            self.head_index = value4 + counter
            self.neckline2_index = value5 + counter
            self.shoulder2_index = value6 + counter
            self.target_index = value7 + counter


            for index in range(self.start_index,self.target_index+1,1):
                csv_data.at[index,'patternName_headandshoulders'] = 'H&S'

            csv_data.at[self.start_index,'patternFeatures_headandshoulders'] = 'START'
            csv_data.at[self.shoulder1_index,'patternFeatures_headandshoulders'] = 'SHOULDER 1'
            csv_data.at[self.neckline1_index,'patternFeatures_headandshoulders'] = 'NECKLINE 1'
            csv_data.at[self.head_index,'patternFeatures_headandshoulders'] = 'HEAD'
            csv_data.at[self.neckline2_index,'patternFeatures_headandshoulders'] = 'NECKLINE 2'
            csv_data.at[self.shoulder2_index,'patternFeatures_headandshoulders'] = 'SHOULDER 2'
            csv_data.at[self.target_index,'patternFeatures_headandshoulders'] = 'TARGET'
            
            self.current_state['total pattern identified'] += 1
            self.current_state['total head and shoulders'] += 1

            #csv_data.to_csv('D:\\market_train\\dataset\\temp_data_creation_updated.csv')




            print(self.start_index,
                self.shoulder1_index,
                self.neckline1_index,
                self.head_index,
                self.neckline2_index,
                self.shoulder2_index,
                self.target_index,
                self.current_pattern)


class process_graph:

    def __init__(self) -> None:
        self.start_price = None
        self.shoulder1_price = None
        self.shoulder2_price = None
        self.neckline_price = None
        self.neckline1_price = None
        self.neckline2_price = None
        self.head_price = None
        self.top1_price = None
        self.top2_price = None
        self.target_price = None
        self.price_counter = 0

    def update_plot(self, counter_increment, prediction = np.ones(100), start_pred = 0, end_pred = 100):
        global counter
        counter += counter_increment
        stock_prices = csv_data[counter:counter+400]

        ax.clear()  

        bar_bottom = min(stock_prices['open'])
        price_difference = max(stock_prices['open']) - min(stock_prices['open'])
        prediction = prediction * price_difference 
        prediction = prediction - prediction * 0.8
        ax.bar(range(start_pred,end_pred),prediction,bottom=min(stock_prices['open']))
        mplf.candlestick2_ohlc(ax, stock_prices['open'], stock_prices['high'], stock_prices['low'], stock_prices['close'],
                                width=0.7, colorup='g', colordown='r')
        canvas.draw()

    def reset(self):
        self.start_price = None
        self.shoulder1_price = None
        self.shoulder2_price = None
        self.neckline_price = None
        self.neckline1_price = None
        self.neckline2_price = None
        self.head_price = None
        self.top1_price = None
        self.top2_price = None
        self.target_price = None
        self.price_counter = 0

    def on_button_press(self,pattern):
        Write_to_csv.current_pattern = pattern
        if pattern == 'DOUBLE TOP':
            Label_changes.change_label_double_top(0)
        elif pattern == 'HEAD AND SHOULDERS':
            Label_changes.change_label_head_and_shoulder(0)
            


    def on_key(self,event):
        if event.key == 'right':
            self.update_plot(40)
            Label_changes.show_counter()
        if event.key == 'left':
            self.update_plot(-20)
            Label_changes.show_counter()

        if event.key == 'shift':
            pred, start, stop = DT_model.get_prediction()
            self.update_plot(0, pred, start, stop)
            Label_changes.show_counter()


    def on_click(self,event):
        x = event.x
        y = event.y

        price_value = (x/4) - 25     
        price_value = int(price_value) 
        
        Label_changes.show_price(price_value)

        if Write_to_csv.current_pattern == 'DOUBLE TOP':
            self.update_double_top_prices(price_value)
            Label_changes.change_label_double_top(self.price_counter)
        elif Write_to_csv.current_pattern == 'HEAD AND SHOULDERS':
            self.update_head_and_shoulder_prices(price_value)
            Label_changes.change_label_head_and_shoulder(self.price_counter)
        


    
    def update_double_top_prices(self,price):

        self.price_counter += 1

        match self.price_counter:
            case 1:
                self.start_price = price
            case 2:
                self.top1_price = price
            case 3:
                self.neckline_price = price
            case 4:
                self.top2_price = price
            case 5:
                self.target_price = price

        if self.price_counter == 5:
            Write_to_csv.write_double_top_to_csv(self.start_price,
                                            self.top1_price,
                                            self.neckline_price,
                                            self.top2_price,
                                            self.target_price)
            self.reset()
            Label_changes.reset()

        #print(self.start_price,self.top1_price,self.neckline_price,self.top2_price,self.target_price,self.price_counter)

    def update_head_and_shoulder_prices(self,price):

        self.price_counter += 1

        match self.price_counter: 
            case 1:
                self.start_price = price
            case 2:
                self.shoulder1_price = price
            case 3:
                self.neckline1_price = price
            case 4:
                self.head_price = price
            case 5:
                self.neckline2_price = price
            case 6:
                self.shoulder2_price = price
            case 7:
                self.target_price = price

        if self.price_counter == 7:
            Write_to_csv.write_head_and_shoulders_to_csv(self.start_price,
                                            self.shoulder1_price,
                                            self.neckline1_price,
                                            self.head_price,
                                            self.neckline2_price,
                                            self.shoulder2_price,
                                            self.target_price)
            self.reset()
            Label_changes.reset()
        #print(self.start_price, self.shoulder1_price,self.neckline1_price,self.head_price,self.neckline2_price,self.shoulder2_price,self.target_price,self.price_counter)


        


class Label_changes:
    def __init__(self) -> None:


        self.price_frame = tk.Frame(app)
        self.price_frame.grid(row=1,column=1,padx=10,pady=10)

        self.price_value_label = ttk.Label(self.price_frame,text=0)
        self.price_value_label.grid(row=0,column=0,padx=10,pady=10)

        self.current_counter_label = ttk.Label(self.price_frame,text=0)
        self.current_counter_label.grid(row=1,column=0,padx=10,pady=10)

        self.clicked_prices_status_frame = tk.Frame(app)
        self.clicked_prices_status_frame.grid(row=1,column=0,padx=10,pady=10)



        self.start = ttk.Label(self.clicked_prices_status_frame,text='START',background='red',foreground='white')
        self.top1 = ttk.Label(self.clicked_prices_status_frame,text='TOP 1',background='red',foreground='white')
        self.neckline = ttk.Label(self.clicked_prices_status_frame,text='NECKLINE',background='red',foreground='white')
        self.top2 = ttk.Label(self.clicked_prices_status_frame,text='TOP 2',background='red',foreground='white')
        self.shoulder1 = ttk.Label(self.clicked_prices_status_frame,text='SHOULDER 1',background='red',foreground='white')
        self.neckline1 = ttk.Label(self.clicked_prices_status_frame,text='NECKLINE 1',background='red',foreground='white')
        self.head = ttk.Label(self.clicked_prices_status_frame,text='HEAD',background='red',foreground='white')
        self.neckline2 = ttk.Label(self.clicked_prices_status_frame,text='NECKLINE 2',background='red',foreground='white')
        self.shoulder2 = ttk.Label(self.clicked_prices_status_frame,text='SHOULDER 2',background='red',foreground='white')
        self.target = ttk.Label(self.clicked_prices_status_frame,text='TARGET',background='red',foreground='white')







    def reset(self):
        self.price_counter = 0

        try:
            self.start.config(background='red')
            self.start.grid_forget()
        except:pass
        try:
            self.shoulder1.config(background='red')
            self.shoulder1.grid_forget() 
        except:pass
        try:
            self.shoulder2.config(background='red')
            self.shoulder2.grid_forget() 
        except:pass
        try:
            self.neckline.config(background='red')
            self.neckline.grid_forget()
        except:pass
        try:
            self.neckline1.config(background='red')
            self.neckline1.grid_forget()
        except:pass
        try:
            self.neckline2.config(background='red')
            self.neckline2.grid_forget()
        except:pass
        try:
            self.head.config(background='red')
            self.head.grid_forget()
        except:pass
        try:
            self.top1.config(background='red')
            self.top1.grid_forget()
        except:pass
        try:
            self.top2.config(background='red')
            self.top2.grid_forget()
        except:pass
        try:
            self.target.config(background='red')
            self.target.grid_forget()
        except:pass



    def change_label_double_top(self,value):
        
        if value == 0:
            self.reset()
            process_graph.reset()
            
            self.start.grid(row=0,column=0,padx=10,pady=10)
            self.top1.grid(row=0,column=1,padx=10,pady=10)
            self.neckline.grid(row=0,column=2,padx=10,pady=10)
            self.top2.grid(row=0,column=3,padx=10,pady=10)
            self.target.grid(row=0,column=4,padx=10,pady=10)
            

        match value:
            case 1:
                self.start.config(background='green')
            case 2:
                self.top1.config(background='green')
            case 3:
                self.neckline.config(background='green')
            case 4:
                self.top2.config(background='green')
            case 5:
                self.target.config(background='green')

    def change_label_head_and_shoulder(self,value):

        if value == 0:
            self.reset()
            process_graph.reset()

            self.start.grid(row=0,column=0,padx=10,pady=10)
            self.shoulder1.grid(row=0,column=1,padx=10,pady=10)
            self.neckline1.grid(row=0,column=2,padx=10,pady=10)
            self.head.grid(row=0,column=3,padx=10,pady=10)
            self.neckline2.grid(row=0,column=4,padx=10,pady=10)
            self.shoulder2.grid(row=0,column=5,padx=10,pady=10)
            self.target.grid(row=0,column=6,padx=10,pady=10)
            

        match value:
            case 1:
                self.start.config(background='green')
            case 2:
                self.shoulder1.config(background='green')
            case 3:
                self.neckline1.config(background='green')
            case 4:
                self.head.config(background='green')
            case 5:
                self.neckline2.config(background='green')
            case 6:
                self.shoulder2.config(background='green')
            case 7:
                self.target.config(background='green')


    def show_price(self,price_value):
        
        self.price_value_label.config(text=price_value,font=('Helvetica bold', 20))
    
    def show_counter(self):
        self.current_counter_label.config(text=counter,font=('Helvetica bold', 20))


class DT_model:
    def __init__(self) -> None:
        self.find_pattern_status = False
        self.stock_prices = None
        self.prediction = None


    def split_and_predict(self, node):

        if node is not None and node.end - node.start > 100 and self.find_pattern_status == False:


            left_value = node.end - ((node.end - node.start) / 2)
            node.left = Tree(node.start, left_value)
            self.split_and_predict(node.left)

            right_value = node.start + ((node.end - node.start) / 2)
            node.right = Tree(right_value, node.end)
            self.split_and_predict(node.right)

            self.find_pattern_status, self.prediction = self.find_pattern(counter + node.start, counter + node.end)

            return self.prediction, node.start, node.end
            

        else:
            return None    

    def find_pattern(self, start, end):
        global model
        stock_prices = np.array(csv_data['close'][int(start):int(end)])
        stock_prices,_ = normalize(stock_prices,len(stock_prices))
        stock_prices = interpolate_sequence(stock_prices, 100)
        stock_prices = np.array(stock_prices)
        stock_prices = stock_prices.reshape(1,100)

        output = model.predict(stock_prices)
        #print(output)
        evaluation,_,_ = self.evaluate_prediction(output.reshape(100))
        if evaluation:
            output = interpolate_sequence(output.reshape(100), int(end - start))
            return True,np.array(output)
        else:
            return False,None
    


    def evaluate_prediction(self,output, min_threshold = 0.5, min_no_of_max = 8):
    
        no_of_maxes = 0
        output_pass = 0
        start = 0
        end = 0
        for index,value in enumerate(output):

            if no_of_maxes >= min_no_of_max:
                output_pass = 1
            if value >= 0.9:
                no_of_maxes += 1
            else:
                if no_of_maxes < min_no_of_max:
                    no_of_maxes = 0
                elif no_of_maxes >= min_no_of_max:
                    end = index 
                    start = index - no_of_maxes 
                    break

        return output_pass,start,end


    def get_prediction(self):
        global counter
        counter += 50
        while counter < len(csv_data):
            self.find_pattern_status = False
            self.prediction = None
            node = Tree(0,400)
            output, start, end = self.split_and_predict(node)

            if self.find_pattern_status:
                print("output passed")
                return output, start, end
            else:
                counter+=100
                print("Can't find pattern, trying again!")
        print("-------------file done--------------")




model = tf.keras.models.load_model("D:\\market_train\\models\\double_top.h5")


counter = 0
start_time = time.time()

app = tk.Tk()
app.title("plot")



fig = Figure(figsize=(18, 8), dpi=100)
fig.subplots_adjust(left=0.01, right=0.99, top=0.95, bottom=0.05)  
ax = fig.add_subplot(111)

canvas = FigureCanvasTkAgg(fig, master=app)
canvas_widget = canvas.get_tk_widget()
canvas_widget.grid(row=0,column=0,padx=10,pady=10)

bottons_frame = tk.Frame(app)
bottons_frame.grid(row=0,column=1,padx=10,pady=10)

Double_top_button = tk.Button(bottons_frame, text="Double top", command=lambda: process_graph.on_button_press('DOUBLE TOP'),height=20,width=10)
Double_top_button.grid(row=0,column=1,padx=10,pady=10)

Head_and_shoulders_button = tk.Button(bottons_frame, text="Head and shoulders", command=lambda: process_graph.on_button_press('HEAD AND SHOULDERS'),height=20,width=10)
Head_and_shoulders_button.grid(row=1, column=1, padx=10, pady=10, sticky="w")

#process_graph.update_plot(0)

Label_changes = Label_changes()
process_graph = process_graph()
Write_to_csv = Write_to_csv()
DT_model = DT_model()


canvas.mpl_connect('key_press_event',process_graph.on_key)
canvas_widget.bind("<Button-1>", process_graph.on_click)

app.mainloop()

In [ ]:
x_data = []
y_data = []

counter = 0
close = csv_data['open']

while True:
    if csv_data['patternName_doubletop'][counter] == 'DT':
        x_temp = []
        print(csv_data['patternName_doubletop'][counter])
        for i in range(counter,len(close)):
            if csv_data['patternName_doubletop'][i] == 'DT':
                x_temp.append(close[i])
            else:
                counter = i
                break
        x_data.append(x_temp)
    counter += 1
    if counter >= len(close):
        break
        



In [ ]:
class Tree:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.left = None
        self.right = None

def traverse(node):

    if node is not None and node.end - node.start >= 100:


        left_value = node.end - ((node.end - node.start) / 2)
        node.left = Tree(node.start, left_value)
        traverse(node.left)

        right_value = node.start + ((node.end - node.start) / 2)
        node.right = Tree(right_value, node.end)
        traverse(node.right)
        
        print(node.start,node.end)

    else:
        return None

node = Tree(0,400)
traverse(node)

In [ ]:
class Tree:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.left = None
        self.right = None


def traverse(node):

    if node is not None and node.end - node.start > 100:


        left_value = node.end - ((node.end - node.start) / 2)
        node.left = Tree(node.start, left_value)
        traverse(node.left)

        right_value = node.start + ((node.end - node.start) / 2)
        node.right = Tree(right_value, node.end)
        traverse(node.right)
        
    else:
        return None



node = Tree(0,800)

traverse(node)

def postorder_traversal(node):
    if node is not None:
        postorder_traversal(node.left)

        # Traverse the right subtree
        postorder_traversal(node.right)

        # Visit the current node (print its value)
        print("node",node.start, node.end)
        
        #if node.start == 400:
        #    return None
        
postorder_traversal(node)

In [1]:
import win32gui
import win32con
import time
import subprocess

def get_open_windows():
    windows = []
    win32gui.EnumWindows(enum_windows_callback, windows)

    for window_name in windows:
        if "NiceHash Miner v" in window_name:
            return window_name


def enum_windows_callback(hwnd, windows):
    window_title = win32gui.GetWindowText(hwnd)
    if window_title:
        windows.append(window_title)


def minimise_window():

    while True:
        window_name = get_open_windows()
        print(window_name)
        hwnd = win32gui.FindWindow(None, window_name)
        print(hwnd)
        if hwnd != 0 and window_name != None:
            time.sleep(0.1)
            win32gui.ShowWindow(hwnd, win32con.SW_FORCEMINIMIZE)
            print("minimised 1")
            break
        else:
            time.sleep(1)

    while True:
        hwnd = win32gui.FindWindow(None, window_name)
        print(hwnd)
        if hwnd != 0:
            time.sleep(1)
            win32gui.ShowWindow(hwnd, win32con.SW_FORCEMINIMIZE)
            print("minimised 2")
            break
        else:
            time.sleep(1)


def open_nicehash():

    path_student = "C:\\Users\\aids\\AppData\\Local\\Programs\\NiceHash Miner\\NiceHashMiner.exe"
    path_aids = "C:\\Users\\lenovo\\AppData\\Local\\Programs\\NiceHash Miner\\NiceHashMiner.exe"


    try:
        subprocess.Popen(path_student)
    except:
        try:
            subprocess.Popen(path_aids)
        except:
            pass
       


if __name__ == "__main__":
    

    minimise_window()

    while True:
        window_name = get_open_windows()
        print(window_name)
        if window_name == None:
            open_nicehash()
            minimise_window()
            

        hwnd = win32gui.FindWindow(None, window_name)

        if hwnd != 0:
            win32gui.ShowWindow(hwnd, win32con.SW_FORCEMINIMIZE)
            print("minimised third")
            time.sleep(20)



    


None
65930
None
65930
None
65930
None
65930
None
65930
None
65930
None
65930
None
65930
NiceHash Miner v3.1.0.9
198764
minimised 1
198764
minimised 2
NiceHash Miner v3.1.0.9
minimised third
NiceHash Miner v3.1.0.9
minimised third
None
None
65930
None
65930
NiceHash Miner v3.1.0.9
721532
minimised 1
721532
minimised 2
minimised third
NiceHash Miner v3.1.0.9
minimised third


In [6]:
import win32gui

def get_open_windows():
    windows = []
    win32gui.EnumWindows(enum_windows_callback, windows)
    print(windows)
    return windows

def enum_windows_callback(hwnd, windows):
    window_title = win32gui.GetWindowText(hwnd)
    if window_title:
        windows.append(window_title)

if __name__ == "__main__":
    open_windows = get_open_windows()

    if open_windows:
        print("Open Windows:")
        for window_title in open_windows:
            #print(window_title)
            pass
    else:
        print("No open windows found.")


['Task Switching', 'System tray overflow window.', 'CiceroUIWndFrame', 'Battery Meter', 'Network Flyout', '● data_creation 2.ipynb - market_train - Visual Studio Code', 'GDI+ Window (app_nhm.exe)', 'CiceroUIWndFrame', 'NiceHash Miner v3.1.0.9', 'MediaContextNotificationWindow', '.NET-BroadcastEventWindow.ecabc2.0', 'SystemResourceNotifyWindow', 'Minimize Window Script - Google Chrome', 'Downloads - Google Chrome', 'FnHotkeyUtility', 'Neat Download Manager 1.2', 'Code', 'DDE Server Window', 'DesktopWindowXamlSource', 'DesktopWindowXamlSource', 'AMD Software: Adrenalin Edition', 'RadeonSoftware', 'RadeonSoftware', 'RadeonSoftware', 'Mail', 'Inbox - Gmail \u200e- Mail', 'Realtek Audio Console', 'Realtek Audio Console', 'DesktopWindowXamlSource', 'Nahimic', 'Nahimic', 'GDI+ Window (PhoneExperienceHost.exe)', '.NET-BroadcastEventWindow.2f3e4cc.0', 'Windows Input Experience', 'AMD:DVR-CapturingWindow', 'GDI+ Window (AMDRSServ.exe)', 'AMD:CCC-AEMCapturingWindow', 'NVIDIA GeForce Overlay DT', 